# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for my team.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplearning-ai
Snippet: Details. Deeplearning.ai is company that explo

Thought: 
I have gathered valuable information from the internet on DeepLearningAI and its recent milestones. I should now compile a detailed report for my team.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI identified needs"} 


Search results: Title: Deep Learning Specialization - DeepLearning.AI
Link: https://www.deeplearning.ai/courses/deep-learning-specialization/
Snippet: The Deep Learning Specialization is a foundational program that will help you understand the capabilities, challenges, and consequences of deep learning and ...
---
Title: Deep Learning AI Needs Tools To Adapt To Changes In The Data ...
Link: https://www.forbes.com/sites/davidteich/2020/07/16/deep-learning-ai-needs-tools-to-adapt-to-changes-in-the-data-environment/
Snippet: Artificial intelligence (AI) is increasingly used to analyze data, and deep learning (DL) is one of the more complex aspects of AI. In multiple ...
---
Title: I have just completed the Deep Learning Specializatio



> Entering new CrewAgentExecutor chain...
I need to craft personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng, in order to demonstrate a deep understanding of their business and needs.

Action: Search the internet
Action Input: {"search_query": "Recent achievements of DeepLearningAI and Andrew Ng"} 


Search results: Title: Andrew Ng: Deepening and democratising learning - AI Magazine
Link: https://aimagazine.com/articles/andrew-ng-deepening-and-democratising-learning
Snippet: His pioneering work in deep learning has propelled AI to new heights, enabling breakthroughs in image and speech recognition, natural language ...
---
Title: Andrew Ng's Next Trick: Training a Million AI Experts
Link: https://www.deeplearning.ai/blog/andrew-ng-training-a-million-ai-experts/
Snippet: Millions of people should master deep learning, says a leading AI researcher and educator. Andrew Ng, one of the world's best-known.
---
Title: Generative AI for Everyone - DeepLea

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Email Draft 1:
Subject: Celebrating Your Milestones at DeepLearningAI

Dear Andrew Ng,

I hope this message finds you well. I recently came across the 256th issue of "The Batch," and I must commend DeepLearningAI for consistently providing valuable insights and updates in the field of AI. Your dedication to making world-class AI education accessible to people globally is truly inspiring.

As a global leader in AI, you have continuously pushed the boundaries of what is possible in the field. Your recent achievements in training a million AI experts and your pioneering work in deep learning have undoubtedly propelled AI to new heights. The impact of your work on image and speech recognition, natural language processing, and machine learning is evident in the success stories shared by your community of AI experts and learners.

At CrewAI, we understand the importance of continuous innovation and adaptation in the rapidly changing data environment. Our solutions are designed to support companies like DeepLearningAI in analyzing data effectively and leveraging the power of AI and deep learning. We believe that our tools can help your community of learners further their knowledge in areas like Computer Vision and advance their skills in AI.

I would love the opportunity to discuss how CrewAI can support DeepLearningAI's mission and contribute to your continued success. Please let me know if you would be open to a conversation at your convenience.

Thank you for your time and consideration.

Warm regards,
[Your Name]
Lead Sales Representative
CrewAI

Email Draft 2:
Subject: Empowering AI Education with CrewAI

Dear Andrew Ng,

I trust this message finds you in good spirits. Your commitment to democratizing AI education and empowering millions of learners around the world is truly commendable. Your vision of making AI accessible and understandable to all has had a profound impact on the field of artificial intelligence.

I recently read about your efforts in launching the "Generative AI for Everyone" course at DeepLearningAI. Your dedication to providing high-quality educational resources to learners of all levels is truly inspiring. The impact of your work in co-founding Google Brain, leading AI at Baidu, and reaching millions of learners through Coursera has transformed the way we approach machine learning and deep learning.

At CrewAI, we share your passion for education and innovation. Our solutions are tailored to support companies like DeepLearningAI in their quest to provide cutting-edge AI education to a global audience. We believe that our tools can help your community of learners enhance their skills in AI and drive meaningful impact in the field.

I would be thrilled to explore how CrewAI can contribute to the success of DeepLearningAI and support your mission of democratizing AI education. If you are open to a conversation, please let me know a time that works best for you.

Thank you for your time and consideration.

Best regards,
[Your Name]
Lead Sales Representative
CrewAI